## Welcome to the Second Lab - Week 1, Day 3

Today we will work with lots of models! This is a way to get comfortable with APIs.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Important point - please read</h2>
            <span style="color:#ff7800;">The way I collaborate with you may be different to other courses you've taken. I prefer not to type code while you watch. Rather, I execute Jupyter Labs, like this, and give you an intuition for what's going on. My suggestion is that you carefully execute this yourself, <b>after</b> watching the lecture. Add print statements to understand what's going on, and then come up with your own variations.<br/><br/>If you have time, I'd love it if you submit a PR for changes in the community_contributions folder - instructions in the resources. Also, if you have a Github account, use this to showcase your variations. Not only is this essential practice, but it demonstrates your skills to others, including perhaps future clients or employers...
            </span>
        </td>
    </tr>
</table>

In [49]:
# Start with imports - ask ChatGPT to explain any package that you don't know

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from IPython.display import Markdown, display

In [50]:
# Always remember to do this!
load_dotenv(override=True)

True

In [51]:
# Print the key prefixes to help with any debugging

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AI
DeepSeek API Key exists and begins sk-
Groq API Key exists and begins gsk_


In [29]:
request = "Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. "
request += "Answer only with the question, no explanation."
messages = [{"role": "user", "content": request}]

In [30]:
messages

[{'role': 'user',
  'content': 'Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. Answer only with the question, no explanation.'}]

In [31]:
openai = OpenAI()
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)
question = response.choices[0].message.content
print(question)


How would you approach the problem of reconciling conflicting ethical principles in a situation where the well-being of one individual directly opposes the rights of a larger group?


In [32]:
competitors = []
answers = []
messages = [{"role": "user", "content": question}]

In [33]:
# The API we know well

model_name = "gpt-4o-mini"

response = openai.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

Reconciling conflicting ethical principles, especially when it involves the well-being of an individual versus the rights of a larger group, is a complex challenge that requires careful consideration and nuanced thinking. Here is a structured approach to such situations:

1. **Identify the Ethical Principles Involved**: Clearly define the principles at stake. For example, this may involve the principle of individual rights (e.g., autonomy, freedom) versus the principle of the greater good (e.g., utilitarianism). Understanding the values and ethical theories behind each side is crucial.

2. **Gather Relevant Information**: Assess the specifics of the situation. This includes understanding the context, the needs and circumstances of both the individual and the group, and any potential consequences of various actions.

3. **Consider Stakeholders’ Perspectives**: Engage with the perspectives of all stakeholders involved—both the individual and representatives of the group. This helps in understanding their needs, values, and concerns.

4. **Explore Compromise Solutions**: Look for ways to reconcile the conflict through compromise or creative solutions that might address both the well-being of the individual and the needs of the group. This could involve finding alternative ways to satisfy the group’s needs while still respecting the individual’s rights.

5. **Evaluate Potential Consequences**: Assess the potential outcomes of various courses of action. This includes short-term and long-term consequences for the individual, the group, and the broader community. Ethical decision-making often involves considering the impact on all parties.

6. **Consult Ethical Frameworks**: Apply ethical frameworks or theories (e.g., consequentialism, deontology, virtue ethics) to analyze the implications of various decisions. This can provide a structured way to weigh the benefits and harms associated with each option.

7. **Seek Consensus**: If possible, involve a deliberative process where stakeholders can discuss and share perspectives. This collaborative approach can lead to greater acceptance of the final decision, even if it doesn't perfectly satisfy all parties.

8. **Review and Reflect**: After a decision is made and implemented, it’s important to review the outcomes and reflect on the ethical implications. This feedback can inform future decisions and improve ethical considerations in similar contexts.

9. **Prioritize Transparency and Accountability**: Ensure that the decision-making process is transparent and that those making the decisions are held accountable. This can help build trust among stakeholders and facilitate future cooperation.

Ultimately, the reconciliation of conflicting ethical principles often requires a balance of respect for individual rights while considering the well-being of the group, aiming for an outcome that is ethically justifiable and maintains dignity for all involved.

In [34]:
# Anthropic has a slightly different API, and Max Tokens is required

model_name = "claude-sonnet-4-5-20250929"

claude = Anthropic()
response = claude.messages.create(model=model_name, messages=messages, max_tokens=1000)
answer = response.content[0].text

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

I'd break this down into several analytical steps:

**First, clarify the actual conflict:**
- Are the principles truly incompatible, or does one take precedence in this context?
- Is "well-being" vs "rights" the real tension, or is it competing rights or different conceptions of well-being?

**Examine the framing:**
- Individual vs. group conflicts often hide false dichotomies
- Sometimes protecting individual rights *is* protecting the group long-term (precedent matters)
- The "larger group" may not be monolithic in its interests

**Consider key factors:**
- **Severity**: Minor convenience for many vs. severe harm to one weighs differently than the reverse
- **Reversibility**: Can we walk back this decision?
- **Consent and voice**: Did affected parties have input?
- **Power dynamics**: Is the "one" already vulnerable or marginalized?

**Practical approaches:**
- Look for creative compromises that reduce the zero-sum nature
- Consider procedural fairness - *how* we decide matters, not just outcomes
- Apply consistency tests: would we accept this reasoning reversed?

**My honest uncertainty:**
These situations rarely have clean answers. I'm suspicious of frameworks that make it *too* easy to sacrifice individuals for collective benefit - history shows how that gets abused. But I also recognize pure individual rights absolutism can be paralyzing.

What's the specific scenario you're wrestling with? The details often reveal which principles genuinely apply.

In [35]:
gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.0-flash"

response = gemini.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

Reconciling conflicting ethical principles when the well-being of one individual opposes the rights of a larger group is a complex ethical dilemma with no easy answer. Here's a multi-faceted approach I would take:

**1. Thorough Analysis of the Situation:**

*   **Identify and Define the Principles in Conflict:**  Specifically, what ethical principles are clashing? Common examples include:
    *   **Individual Well-being vs. Collective Good:** The rights of the individual to autonomy, protection, and fair treatment versus the overall benefits for the majority.
    *   **Justice vs. Mercy:** Upholding fairness and rules for everyone versus making exceptions based on individual circumstances and compassion.
    *   **Beneficence vs. Non-maleficence:**  Doing good for one person versus avoiding harm to the larger group.
    *   **Rights-based ethics:** Which set of rights is being prioritized? Individual rights (privacy, bodily autonomy) versus collective rights (public safety, national security).
*   **Gather All Relevant Facts:**  What are the specific circumstances? What are the potential consequences of different actions?  What evidence supports these consequences? Avoid relying on assumptions or biases.
*   **Identify All Stakeholders:**  Who is directly and indirectly affected by the decision?  Consider the perspectives of the individual, the group, and any secondary parties involved.
*   **Assess the Urgency and Severity:** How immediate is the situation? What is the potential magnitude of harm to both the individual and the group?

**2. Ethical Frameworks & Theories:**

Apply different ethical frameworks to analyze the situation from multiple angles. No single framework provides all the answers, but considering several can illuminate different aspects of the problem:

*   **Utilitarianism:**  Focuses on maximizing overall happiness and minimizing suffering.  Weigh the potential benefits and harms to the individual and the group. Which action produces the greatest net good for the greatest number?  (Problem: Can potentially justify sacrificing individual rights for the sake of the majority).
*   **Deontology (Duty-Based Ethics):**  Emphasizes moral duties and rules, regardless of consequences.  Are there any universal moral rules that apply, such as respecting autonomy or avoiding harm?  (Problem:  Can be rigid and lead to undesirable outcomes in specific situations).
*   **Rights-Based Ethics:**  Focuses on inherent rights and freedoms of individuals.  Which rights are at stake? How can we best protect those rights while minimizing harm? (Problem: Rights can conflict, and prioritizing one set of rights may infringe upon others).
*   **Virtue Ethics:**  Focuses on developing virtuous character traits, such as compassion, justice, and integrity.  What would a virtuous person do in this situation? (Problem:  Virtue ethics can be subjective and difficult to apply consistently).
*   **Care Ethics:**  Emphasizes relationships, empathy, and responsiveness to the needs of others.  How can we best care for both the individual and the group, considering their specific vulnerabilities and needs?  (Problem:  Can be seen as favoring personal relationships over impartial justice).

**3. Possible Solutions and Their Consequences:**

*   **Brainstorm a Range of Options:**  Don't limit yourself to obvious choices.  Think creatively about potential solutions that might partially satisfy the needs of both the individual and the group.
*   **Evaluate Each Option:**  Carefully analyze the potential consequences of each option for the individual, the group, and other stakeholders. Use the ethical frameworks to guide your evaluation. Consider both short-term and long-term effects.
*   **Look for Compromise and Mitigation:**  Are there ways to mitigate the harm to either the individual or the group? Can a compromise be reached that balances the competing interests?  Can resources be allocated to support those who are negatively affected?

**4. Decision-Making and Justification:**

*   **Transparency and Stakeholder Involvement:**  Involve stakeholders in the decision-making process, if possible.  Explain the ethical considerations, the analysis, and the reasoning behind the chosen course of action.  This promotes trust and understanding, even if not everyone agrees.
*   **Justification and Explanation:**  Be prepared to justify your decision based on ethical principles and the analysis of the situation.  Clearly articulate why you believe this is the most ethical course of action, given the circumstances.
*   **Documentation:** Keep a detailed record of the situation, the ethical analysis, the options considered, the rationale for the decision, and the potential consequences. This documentation is crucial for accountability and learning.
*   **Ongoing Monitoring and Adjustment:**  The situation may evolve over time. Continuously monitor the consequences of the decision and be prepared to adjust course if necessary.

**Specific Considerations in Different Scenarios:**

The best approach will vary depending on the specific situation. Here are some considerations for common scenarios:

*   **Public Health vs. Individual Liberty:** Balancing public health measures (e.g., mandatory vaccinations) with individual rights to bodily autonomy requires careful consideration of the severity of the threat, the effectiveness of the intervention, and the least restrictive means of achieving the desired outcome.
*   **National Security vs. Privacy:**  Protecting national security (e.g., surveillance programs) can conflict with individual rights to privacy.  Weigh the potential benefits of security measures against the potential harms to civil liberties.  Implement robust oversight mechanisms to prevent abuse.
*   **Resource Allocation (e.g., Healthcare):**  Limited resources often force difficult choices about who receives treatment.  Consider principles of fairness, need, and the potential for benefit.  Use transparent and equitable decision-making processes.

**Important Considerations:**

*   **Objectivity and Bias:**  Be aware of your own biases and preconceptions, and strive for objectivity in your analysis. Seek input from diverse perspectives to challenge your assumptions.
*   **Emotional Intelligence:**  Acknowledge and manage your own emotions, as well as the emotions of others involved.  Approach the situation with empathy and compassion.
*   **Fallibility:**  Recognize that you may not have all the answers, and that even the best decisions can have unintended consequences. Be open to learning and adapting your approach as needed.

In conclusion, navigating these complex ethical dilemmas requires a careful, systematic, and thoughtful approach. By applying ethical frameworks, considering the perspectives of all stakeholders, and focusing on transparency and justification, we can strive to make the most ethical decision possible, even when faced with seemingly irreconcilable conflicts.  There is rarely a perfect solution, but a well-reasoned and ethically grounded approach is essential.


In [36]:
deepseek = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com/v1")
model_name = "deepseek-chat"

response = deepseek.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

This is a classic and profound ethical dilemma — essentially a tension between **utilitarian** (greatest good for the greatest number) and **deontological** (rights/duties-based) frameworks.  

Here’s a structured approach to reconciling such conflicts:

---

### **1. Clarify the specifics**
- **Identify the principles in conflict:**  
  For example: individual autonomy vs. public safety, privacy vs. security, one person’s survival vs. many people’s moderate benefit.
- **Gather facts:**  
  What exactly is the threat to the individual’s well-being? What rights of the group are at stake? Are there alternative ways to satisfy both sides partially?
- **Determine irreconcilability:**  
  Is this a true zero-sum conflict, or can creative solutions satisfy core interests of both?

---

### **2. Ethical analysis frameworks**
- **Principle-based approach** (Beauchamp & Childress):  
  Balance *autonomy, beneficence, non-maleficence, justice*.  
  Ask: Can we minimize harm to the individual while protecting group rights? Is the burden on the individual proportionate?
- **Rights-based perspective:**  
  Some rights (e.g., against torture, basic dignity) might be considered inviolable regardless of group benefit.  
  Determine if group “rights” are truly fundamental rights or rather aggregated interests.
- **Consequentialist analysis:**  
  Weigh outcomes — but not just numbers; consider intensity of harm/benefit, long-term precedent, and secondary effects (e.g., trust erosion if individual rights are overridden).
- **Virtue ethics consideration:**  
  What would a wise, compassionate, just person do? This focuses on moral character and context.

---

### **3. Seek integration or prioritization**
- **Hierarchy of principles:**  
  Some ethical systems prioritize certain principles (e.g., in medicine, autonomy often trumps beneficence unless imminent grave public harm).
- **Proportionality test:**  
  Is the infringement on the individual’s well-being proportional to the harm prevented to the group?  
  Is it the least restrictive alternative?
- **Procedural fairness:**  
  Ensure the decision process is transparent, inclusive, and allows voice for the affected individual.

---

### **4. Mitigation and compensation**
- If the group’s rights must prevail, can society compensate or support the individual to offset the harm?
- If the individual’s well-being is prioritized, can we mitigate negative group impacts through other means?

---

### **5. Reflect on precedent and rule-setting**
- Consider the broader impact of the decision as a precedent:  
  “What if everyone acted this way?” or “What kind of society do we create if we routinely sacrifice individuals for groups?”
- Check consistency with existing laws and social contracts.

---

### **6. Practical example: Public health vs. individual liberty**
During a pandemic, mandatory quarantine (infringing individual freedom) to protect public health (group well-being) is often justified if:
- The threat is serious and evidence-based.
- The measure is necessary and effective.
- It’s applied fairly and with time limits.
- Support is provided to those restricted.

---

### **Conclusion**
There’s no universal formula, but a **fair, reflective process** that:
1. **Seeks common ground** first,
2. **Minimizes harm**,
3. **Respects fundamental human dignity** even when limiting some interests,
4. **Remains open to revising** the decision as circumstances change,

…is often the most ethically defensible path.

Would you like to explore a specific scenario where these principles clash?

In [37]:
groq = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
model_name = "llama-3.3-70b-versatile"

response = groq.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)


Reconciling conflicting ethical principles in situations where individual well-being opposes group rights requires a nuanced and multi-step approach. Here's a framework to consider:

1. **Identify the conflicting principles**: Clearly articulate the ethical principles at play, such as individual autonomy, dignity, or well-being, and the rights of the larger group, like equality, fairness, or protection.
2. **Define the context and stakeholders**: Understand the specific situation, the individuals or groups involved, and their interests, values, and perspectives. Consider the power dynamics, social and cultural norms, and any historical or systemic factors that may influence the situation.
3. **Apply ethical theories and frameworks**: Draw on various ethical theories, such as:
	* Utilitarianism: Maximize overall well-being or happiness.
	* Deontology: Respect individual rights and duties, regardless of consequences.
	* Virtue ethics: Consider the character and moral virtues of the individuals involved.
	* Care ethics: Emphasize empathy, care, and compassion in relationships.
4. **Evaluate the moral implications**: Assess the potential consequences of each possible course of action, considering both short-term and long-term effects on the individual and the group. Weigh the moral importance of each principle and the potential harm or benefits of compromising one principle over another.
5. **Consider the concept of proportionality**: Balance the individual's well-being against the group's rights, ensuring that the measures taken are proportionate to the situation and do not disproportionately harm either party.
6. **Seek common ground and compromise**: Look for areas of overlap or common interests between the individual and the group. Explore possible compromises or alternative solutions that can satisfy both parties to some extent.
7. **Apply the principle of minimal harm**: When compromises are not possible, aim to minimize harm to all parties involved. Prioritize the most vulnerable or disadvantaged individuals or groups.
8. **Engage in open communication and dialogue**: Foster an open and respectful exchange of ideas, concerns, and values among all stakeholders. This can help build trust, clarify misunderstandings, and facilitate cooperation.
9. **Reflect on power dynamics and privilege**: Acknowledge and address any power imbalances or privilege that may influence the situation. Strive to create a more equitable and just environment.
10. **Re-evaluate and adjust**: Continuously monitor the situation and be prepared to adjust your approach as new information arises or circumstances change.

Some additional principles to keep in mind:

* **The precautionary principle**: When in doubt, prioritize caution and avoid actions that may cause harm.
* **The principle of non-maleficence**: Avoid causing harm or injury to individuals or groups.
* **The principle of beneficence**: Act in ways that promote the well-being and flourishing of individuals and groups.

Ultimately, reconciling conflicting ethical principles requires a thoughtful, nuanced, and context-sensitive approach. By considering multiple perspectives, evaluating moral implications, and seeking common ground, you can strive to find a resolution that respects the dignity and well-being of all parties involved.

## For the next cell, we will use Ollama

Ollama runs a local web service that gives an OpenAI compatible endpoint,  
and runs models locally using high performance C++ code.

If you don't have Ollama, install it here by visiting https://ollama.com then pressing Download and following the instructions.

After it's installed, you should be able to visit here: http://localhost:11434 and see the message "Ollama is running"

You might need to restart Cursor (and maybe reboot). Then open a Terminal (control+\`) and run `ollama serve`

Useful Ollama commands (run these in the terminal, or with an exclamation mark in this notebook):

`ollama pull <model_name>` downloads a model locally  
`ollama ls` lists all the models you've downloaded  
`ollama rm <model_name>` deletes the specified model from your downloads

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Super important - ignore me at your peril!</h2>
            <span style="color:#ff7800;">The model called <b>llama3.3</b> is FAR too large for home computers - it's not intended for personal computing and will consume all your resources! Stick with the nicely sized <b>llama3.2</b> or <b>llama3.2:1b</b> and if you want larger, try llama3.1 or smaller variants of Qwen, Gemma, Phi or DeepSeek. See the <A href="https://ollama.com/models">the Ollama models page</a> for a full list of models and sizes.
            </span>
        </td>
    </tr>
</table>

In [38]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [39]:
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
model_name = "llama3.2"

response = ollama.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

Reconciling conflicting ethical principles in situations where the well-being of an individual directly opposes the rights of a larger group can be a complex and nuanced challenge. Here's a step-by-step approach to help address this dilemma:

1. **Acknowledge the Conflicting Values**: Recognize the existence of two or more competing moral values, namely the right to autonomy/well-being of the individual and the rights/cultural/traditional significance of the larger group.
2. **Understand the Context**: Gather as much information as possible about the situation, including the specific circumstances, relevant historical and cultural background, and any applicable laws or regulations.
3. **Identify the Key Stakeholders**: Determine who has a vested interest in the outcome, including the individual affected, the larger group, and any other relevant parties such as family members, healthcare providers, social workers, or community leaders.
4. **Examine Relevant Principles and Theories**: Consider the ethical principles and theories that apply to the situation, such as:
	* Autonomy: respect for an individual's right to make decisions about their own life.
	* Beneficence: doing good and promoting the well-being of others.
	* Non-maleficence (do no harm): avoiding causing harm or suffering to others.
	* Justice: fair treatment and distribution of resources.
5. **Consider Alternative Perspectives**: Seek input from diverse perspectives, including those of experts in relevant fields, such as sociology, anthropology, public health, human rights, and ethics.
6. **Weigh the Competing Values**: Use a decision-making framework to systematically evaluate the relative importance of each value. For instance:
	* Create a pros and cons table or matrix to compare the potential outcomes of different courses of action.
	* Ask yourself questions like: "What are the long-term consequences of choosing one option over another?"
	* Consider any mitigating factors that may impact the chosen course of action.
7. **Look for Common Ground**: Seek areas of agreement and potential compromises between the competing values. For example:
	* Could you find ways to meet both the individual's needs (autonomy) and the larger group's rights/fairness?
	* Are there alternative solutions that could balance competing interests, such as mediation or dispute resolution processes?
8. **Seek External Guidance**: Consult relevant authorities, such as courts, regulatory agencies, or independent review bodies, if necessary, to gain insight into applicable laws and policies.
9. **Develop a Personal Position**: Based on the evaluation of relevant factors, formulate a position statement that reflects your values and priorities.
10. **Consider Consequences**: Anticipate potential outcomes from your chosen course of action and be prepared for ongoing monitoring and adaptation as needed.

Some additional strategies to help reconcile conflicting ethical principles in this situation include:

* **Reframing perspectives**: Try to see the world from different viewpoints, even if they are not shared by everyone.
* **Emphasizing moral courage**: Encourage individuals to have courageously held their values even when faced with difficult choices.
* **Celebrating diverse successes**: Emphasize that what's considered most effective in one situation may not be applicable in another
Lastly, recognize that reconciling conflicting ethical principles is often an iterative and context-dependent process. Be willing to revise your analysis as new information or perspectives arise.

This multi-step approach can help you better navigate the complexities of reconciling competing values and make decisions that prioritize both individual well-being and group rights.

In [40]:
# So where are we?

print(competitors)
print(answers)


['gpt-4o-mini', 'claude-sonnet-4-5-20250929', 'gemini-2.0-flash', 'deepseek-chat', 'llama-3.3-70b-versatile', 'llama3.2']
["Reconciling conflicting ethical principles, especially when it involves the well-being of an individual versus the rights of a larger group, is a complex challenge that requires careful consideration and nuanced thinking. Here is a structured approach to such situations:\n\n1. **Identify the Ethical Principles Involved**: Clearly define the principles at stake. For example, this may involve the principle of individual rights (e.g., autonomy, freedom) versus the principle of the greater good (e.g., utilitarianism). Understanding the values and ethical theories behind each side is crucial.\n\n2. **Gather Relevant Information**: Assess the specifics of the situation. This includes understanding the context, the needs and circumstances of both the individual and the group, and any potential consequences of various actions.\n\n3. **Consider Stakeholders’ Perspectives**

In [41]:
# It's nice to know how to use "zip"
for competitor, answer in zip(competitors, answers):
    print(f"Competitor: {competitor}\n\n{answer}")


Competitor: gpt-4o-mini

Reconciling conflicting ethical principles, especially when it involves the well-being of an individual versus the rights of a larger group, is a complex challenge that requires careful consideration and nuanced thinking. Here is a structured approach to such situations:

1. **Identify the Ethical Principles Involved**: Clearly define the principles at stake. For example, this may involve the principle of individual rights (e.g., autonomy, freedom) versus the principle of the greater good (e.g., utilitarianism). Understanding the values and ethical theories behind each side is crucial.

2. **Gather Relevant Information**: Assess the specifics of the situation. This includes understanding the context, the needs and circumstances of both the individual and the group, and any potential consequences of various actions.

3. **Consider Stakeholders’ Perspectives**: Engage with the perspectives of all stakeholders involved—both the individual and representatives of th

In [42]:
# Let's bring this together - note the use of "enumerate"

together = ""
for index, answer in enumerate(answers):
    together += f"# Response from competitor {index+1}\n\n"
    together += answer + "\n\n"

In [43]:
print(together)

# Response from competitor 1

Reconciling conflicting ethical principles, especially when it involves the well-being of an individual versus the rights of a larger group, is a complex challenge that requires careful consideration and nuanced thinking. Here is a structured approach to such situations:

1. **Identify the Ethical Principles Involved**: Clearly define the principles at stake. For example, this may involve the principle of individual rights (e.g., autonomy, freedom) versus the principle of the greater good (e.g., utilitarianism). Understanding the values and ethical theories behind each side is crucial.

2. **Gather Relevant Information**: Assess the specifics of the situation. This includes understanding the context, the needs and circumstances of both the individual and the group, and any potential consequences of various actions.

3. **Consider Stakeholders’ Perspectives**: Engage with the perspectives of all stakeholders involved—both the individual and representatives 

In [44]:
judge = f"""You are judging a competition between {len(competitors)} competitors.
Each model has been given this question:

{question}

Your job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.
Respond with JSON, and only JSON, with the following format:
{{"results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}}

Here are the responses from each competitor:

{together}

Now respond with the JSON with the ranked order of the competitors, nothing else. Do not include markdown formatting or code blocks."""


In [45]:
print(judge)

You are judging a competition between 6 competitors.
Each model has been given this question:

How would you approach the problem of reconciling conflicting ethical principles in a situation where the well-being of one individual directly opposes the rights of a larger group?

Your job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.
Respond with JSON, and only JSON, with the following format:
{"results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}

Here are the responses from each competitor:

# Response from competitor 1

Reconciling conflicting ethical principles, especially when it involves the well-being of an individual versus the rights of a larger group, is a complex challenge that requires careful consideration and nuanced thinking. Here is a structured approach to such situations:

1. **Identify the Ethical Principles Involved**: Clearly define the principles at st

In [46]:
judge_messages = [{"role": "user", "content": judge}]

In [47]:
# Judgement time!

openai = OpenAI()
response = openai.chat.completions.create(
    model="o3-mini",
    messages=judge_messages,
)
results = response.choices[0].message.content
print(results)


{"results": ["3", "4", "1", "5", "6", "2"]}


In [48]:
# OK let's turn this into results!

results_dict = json.loads(results)
ranks = results_dict["results"]
for index, result in enumerate(ranks):
    competitor = competitors[int(result)-1]
    print(f"Rank {index+1}: {competitor}")

Rank 1: gemini-2.0-flash
Rank 2: deepseek-chat
Rank 3: gpt-4o-mini
Rank 4: llama-3.3-70b-versatile
Rank 5: llama3.2
Rank 6: claude-sonnet-4-5-20250929


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Which pattern(s) did this use? Try updating this to add another Agentic design pattern.
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">These kinds of patterns - to send a task to multiple models, and evaluate results,
            are common where you need to improve the quality of your LLM response. This approach can be universally applied
            to business projects where accuracy is critical.
            </span>
        </td>
    </tr>
</table>